# SC1015 Mini-Project: Spotify music recommender system

### By group _

## Our problem statement

For our project, we will be using the Spotify Dataset on kaggle:  for training our models.

Our problem statement for this project would be 

## Data Preparation and Cleaning

In [4]:
pip install spotipy

  Obtaining dependency information for spotipy from https://files.pythonhosted.org/packages/b8/e8/4c099f9431ec9a86f576b344702cd4446d1ff7df09b172dc1951f25d58b1/spotipy-2.23.0-py3-none-any.whl.metadata
  Obtaining dependency information for redis>=3.5.3 from https://files.pythonhosted.org/packages/bb/f1/a384c5582d9a28e4a02eb1a2c279668053dd09aafeb08d2bd4dd323fc466/redis-5.0.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 kB 14.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [14]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

# Libraries to be used later
import spotipy

In [21]:
dataset1_df = pd.read_csv('dataset.csv')
print(dataset1_df.shape)


try:
    dataset2_df = pd.read_csv('spotify2023.csv', encoding='utf-8')
except UnicodeDecodeError:
    try:
        dataset2_df = pd.read_csv('spotify2023.csv', encoding='latin-1')
    except UnicodeDecodeError:
        dataset2_df = pd.read_csv('spotify2023.csv', encoding='cp1252')
print(dataset2_df.shape)

(114000, 21)
(953, 24)


In [17]:
columns_to_drop = ['track_id', 'album_name','explicit','time_signature','track_genre']
dataset1_df.drop(columns_to_drop, axis=1, inplace=True)
dataset1_df.head()

,Unnamed: 0,artists,track_name,popularity,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0,Gen Hoshino,Comedy,73,230666,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917
1,1,Ben Woodward,Ghost - Acoustic,55,149610,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489
2,2,Ingrid Michaelson;ZAYN,To Begin Again,57,210826,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332
3,3,Kina Grannis,Can't Help Falling In Love,71,201933,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740
4,4,Chord Overstreet,Hold On,82,198853,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949


## Exploratory Data Analysis

## Our Models

## Data-driven insights and the recommendations